# Prolema del commesso viaggiatore (TSP) Tabu search
#### il viaggiatore deve visitare ogni città del suo territorio almeno una volta e poi tornare al punto di partenza percorrendola distanza più veloce
#### Il problema è così definito

- Stati: un qualsiasi percorso che includa una sola volta tuttele n città. Possiamo rappresentare uno stato con una lista dicittà lunga n.
- Funzione successore: un qualsiasi scambio di due città.
- Funzione di valutazione: calcolo lunghezza percorso.
- Stato iniziale: una lista di città lunga n.

In [5]:
import copy

#connessioni tra le cità con le relative distanze
connections = {}

connections['A'] = [['B', 4], ['C', 2], ['D',  1], ['E',  2]]
connections['B'] = [['A', 4], ['C', 1], ['D', 2], ['E', 3]]
connections['C'] = [['A', 2], ['B', 1], ['D', 3], ['E',  2]]
connections['D'] = [['A', 1], ['B', 2], ['C',  3], ['E',  2]]
connections['E'] = [['A', 2], ['B',  3], ['C',  2], ['D',  2]]

#Valutazione di uno stato in base al suo percorso
def eval_stato(stato):
        
    lunghezza_percorso = 0
    for i in range(len(stato)):
        l = connections[stato[i]]
        if i < (len(stato) - 1):
            nodo_succ = stato[i + 1]
        else:
            nodo_succ = stato[0]
        for (successore, distanza) in l:
            if successore == nodo_succ:
                lunghezza_percorso += distanza
                break
    return(lunghezza_percorso)

#Funzione per la generazione dei successori di uno stato, l'output è una lista

def generazione_successori(stato): 
    """"
    genera la lista ordinata di successori di uno stato
    """""
        
    lista = []
    t = len(stato)
    
    #Dalla lunghezza della lista stato inverti pseudocasualmente le città e valutale
    for i in range(0, t-1):
        for j in range(i+1, t):
            buffer = copy.deepcopy(stato)
            temp = buffer[i]
            buffer[i] = buffer[j]
            buffer[j] = temp
            eval_successore = eval_stato(buffer)
            lista.append((buffer, eval_successore, (stato[i], stato[j]))) #il tutto è riportato in questa lista, dove:
            #- buffer rappresenta il nuovo stato modificato
            #- eval_successore rappresenta la valutazione del buffer corrente
            #- (stato[i], stato[j]) rappresentano lo scambio delle città effettuato 

    lista.sort(key=lambda x: x[1])  # ordiniamo i successori in base alla loro valutazione eval_stato
    return(lista)
    
#Verifica se è possibile fare una determinata mossa, se non presente nella tabu_list

def tabu_test(sequenza, tabu_list):    # è True se una mossa NON è presente nella tabu_list
    
    a, b = sequenza[2]
    if ((a, b) in tabu_list or (b, a) in tabu_list):
        assente = False
    else:
        assente = True
    return(assente)
            
#Algoritmo di tabu search

def tabu_search(stato_iniziale, tabu_tenure, connections):

    #impostazione dello stato iniziale 
    current = stato_iniziale #definisco lo stato iniziale
    eval_current = eval_stato(current) #definisco la valutazione dello stato iniziale

    #inizializzo la miglior soluzione (best)
    best = stato_iniziale
    eval_best = eval_current

    tabu_list = {} #inizializzo la lista tabu
    neighbours = [] #lista dei vicini

    cont = 0

    #while not criterio di terminazione (preso a piacere)
    while cont < 10:
        
        #genera i successori (stato , eval_stato, mossa effettuata) e ordiniamo la lista sulla valutazione dello stato
        lista_successori = generazione_successori(current)
        
        #seleziona successori non tabu
        #questa è semplicemente la lista dei successori filtrata con i non tabu attraverso la funz. filter
        '''Data una lista di sucessori, la funzione filter(if -> boolean, lista da iterare), seleziona
        gli elementi iterati che soddisfano la tabu_list quindi se non presenti restituiscono true'''
        neighbours = list(filter(lambda n: tabu_test(n, tabu_list), lista_successori)) 
        
        #versione non filter
        '''for i in lista_successori:
            if(tabu_test(i, tabu_list)):
                neighbours.append(i)'''

        #selezione del migliore dei successori
        next_state = neighbours[0][0]
        eval_next_state = neighbours[0][1]
        delta = eval_best - eval_next_state
        if delta > 0: #se la valutazione dello stato migliore è maggiore del successivo
            best = next_state
            eval_best = eval_next_state
        current = next_state
        eval_current = eval_next_state

        #decremento del tabu_tenure dalla tabu_list
        for mossa in tabu_list:
            tabu_list[mossa] = tabu_list[mossa] - 1

        #eliminazione elementi con tenure uguale a zero
        #primo metodo
        #tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k]!= 0} 
        #riformula la tabu_list con i nuovi valori ed elimina nel caso quelli uguale a zero
        #secondo metodo

        temp = {}
        for k in tabu_list:
            if tabu_list[k]!=0:
                temp[k] = tabu_list[k]
        
        tabu_list = temp

        #inserimento della mossa di next nella tabu_list
        mossa_next = neighbours[0][2]
        tabu_list[mossa_next] = tabu_tenure

        cont += 1

    return(best + [best[0]], eval_best)

stato = ['A', 'B', 'C', 'D', 'E']
tabu_search(stato, 3, connections)




        
        
        



(['C', 'B', 'D', 'A', 'E', 'C'], 8)

# Prolema del commesso viaggiatore (TSP) Simulated Annealing
#### il viaggiatore deve visitare ogni città del suo territorio almeno una volta e poi tornare al punto di partenza percorrendola distanza più veloce
#### Il problema è così definito

- Stati: un qualsiasi percorso che includa una sola volta tuttele n città. Possiamo rappresentare uno stato con una lista dicittà lunga n.
- Funzione successore: un qualsiasi scambio di due città.
- Funzione di valutazione: calcolo lunghezza percorso.
- Stato iniziale: una lista di città lunga n.

In [25]:
import random
import copy
import math

#connessioni tra le cità con le relative distanze
connections = {}

connections['A'] = [['B', 4], ['C', 2], ['D',  1], ['E',  2]]
connections['B'] = [['A', 4], ['C', 1], ['D', 2], ['E', 3]]
connections['C'] = [['A', 2], ['B', 1], ['D', 3], ['E',  2]]
connections['D'] = [['A', 1], ['B', 2], ['C',  3], ['E',  2]]
connections['E'] = [['A', 2], ['B',  3], ['C',  2], ['D',  2]]

#Valutazione di uno stato in base al suo percorso
def eval_stato(stato):
        
    lunghezza_percorso = 0
    for i in range(len(stato)):
        l = connections[stato[i]]
        if i < (len(stato) - 1):
            nodo_succ = stato[i + 1]
        else:
            nodo_succ = stato[0]
        for (successore, distanza) in l:
            if successore == nodo_succ:
                lunghezza_percorso += distanza
                break
    return(lunghezza_percorso)

#Funzione per la generazione dei successori di uno stato, l'output è una lista

def generazione_successori(stato): 
    """"
    genera la lista ordinata di successori di uno stato
    """""
        
    lista = []
    t = len(stato)
    
    #Dalla lunghezza della lista stato inverti pseudocasualmente le città e valutale
    for i in range(0, t-1):
        for j in range(i+1, t):
            buffer = copy.deepcopy(stato)
            temp = buffer[i]
            buffer[i] = buffer[j]
            buffer[j] = temp
            eval_successore = eval_stato(buffer)
            lista.append((buffer, eval_successore, (stato[i], stato[j]))) #il tutto è riportato in questa lista, dove:
            #- buffer rappresenta il nuovo stato modificato
            #- eval_successore rappresenta la valutazione del buffer corrente
            #- (stato[i], stato[j]) rappresentano lo scambio delle città effettuato 

    lista.sort(key=lambda x: x[1])  # ordiniamo i successori in base alla loro valutazione eval_stato
    return(lista)
        

#Algoritmo di Simulated Annealing
def simulated_annealing(stato):
    current = stato
    current_val = eval_stato(current)
    next = None
    best = current
    best_val = current_val
    t = TEMPERATURA_INIZIALE
    

    while(t>TEMPERATURA_FINALE and current_val != 8):
        n=3
        while(n>0):
            successori = generazione_successori(current)
            next = successori[0][0]
            next_val = successori[0][1]
            metropolis = math.exp(-(current_val - next_val)/t)
            if next_val < current_val:
                current = next
                current_val = next_val
                if current_val < best_val:
                    best = current
                    best_val = current_val
            
            elif random.random() < metropolis:
                current = next
            n = n-1
    t = t * 0.99
    return (best + list(best[0]), best_val)

stato = ['A', 'B', 'C', 'D', 'E']
TEMPERATURA_INIZIALE = 10
TEMPERATURA_FINALE = 0.2
print(simulated_annealing(stato))



(['C', 'B', 'D', 'A', 'E', 'C'], 8)
